In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from fastai.vision.all import *
import fastai
import timm
import torch.nn as nn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
import os
import glob
#from torch.optim import Adam
from torch.optim import SGD
from torch.autograd import Variable
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image

In [2]:
train_transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(), 
    ])

test_transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    ])

In [3]:
train_path='/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/train'
test_path='/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/val'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=train_transformer),
    batch_size=32, shuffle=True
)

test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=test_transformer),
    batch_size=32, shuffle=True
)

**Part 1:** Applying KNN and Naive Bayes Classifier to Images' Pixel Values

In [4]:
#Extracting features and labels from data loaders
x_train = torch.cat([batch[0].view(batch[0].size(0), -1) for batch in train_loader]).numpy()
y_train = torch.cat([batch[1] for batch in train_loader]).numpy()
x_test = torch.cat([batch[0].view(batch[0].size(0), -1) for batch in test_loader]).numpy()
y_test = torch.cat([batch[1] for batch in test_loader]).numpy()

In [5]:
#Feature scaling (necessary for KNN)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [6]:
# K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train_scaled, y_train)
knn_pred = knn.predict(x_test_scaled)

In [7]:
#Calculating metrics for KNN
knn_accuracy = accuracy_score(y_test, knn_pred)
knn_precision = precision_score(y_test, knn_pred, average='weighted')
knn_recall = recall_score(y_test, knn_pred, average='weighted')

print("KNN Accuracy:", knn_accuracy)
print("KNN Precision:", knn_precision)
print("KNN Recall:", knn_recall)

KNN Accuracy: 0.5033333333333333
KNN Precision: 0.5166964285714286
KNN Recall: 0.5033333333333333


In [8]:
# Naive Bayes
nb = GaussianNB()
nb.fit(x_train, y_train)
nb_pred = nb.predict(x_test)

In [9]:
#Calculating metrics for Naive Bayes
nb_accuracy = accuracy_score(y_test, nb_pred)
nb_precision = precision_score(y_test, nb_pred, average='weighted')
nb_recall = recall_score(y_test, nb_pred, average='weighted')

print("Naive Bayes Accuracy:", nb_accuracy)
print("Naive Bayes Precision:", nb_precision)
print("Naive Bayes Recall:", nb_recall)

Naive Bayes Accuracy: 0.47
Naive Bayes Precision: 0.49375
Naive Bayes Recall: 0.47


**Part 2:** Performing Clasiification using Transfer Learning

In [10]:
path = Path('/kaggle/input/bone-fracture-detection-using-xrays/archive (6)')

In [11]:
dls = ImageDataLoaders.from_folder(path, train='train', valid_pct=0.2 , bs=32, item_tfms=Resize(150, ResizeMethod.Squish),seed=42)

In [12]:
resnet_model = timm.create_model('resnet50', pretrained=True, num_classes=2)
for param in resnet_model.parameters():
    param.requires_grad = True

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [13]:
save_callback = SaveModelCallback(monitor='accuracy', fname='resnet50_best_model')

In [14]:
learner = Learner(dls = dls, model = resnet_model, model_dir = "/kaggle/working/", metrics = [accuracy, error_rate], cbs=save_callback,loss_func=CrossEntropyLossFlat(), opt_func=Adam)

In [15]:
learner.fit_one_cycle(10, lr_max = 0.001)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.395679,0.212595,0.911734,0.088266,00:32
1,0.096578,0.106647,0.962474,0.037526,00:31
2,0.049002,0.066886,0.978330,0.021670,00:30
3,0.032229,0.939639,0.744186,0.255814,00:31
4,0.020847,0.006614,0.998414,0.001586,00:31
5,0.016578,0.006263,0.997886,0.002114,00:30
6,0.002309,0.003163,0.997886,0.002114,00:30
7,0.000304,0.000587,0.999471,0.000529,00:30
8,0.000360,0.000516,0.999471,0.000529,00:31
9,0.000247,0.000542,0.999471,0.000529,00:31


Better model found at epoch 0 with accuracy value: 0.9117336273193359.
Better model found at epoch 1 with accuracy value: 0.9624735713005066.
Better model found at epoch 2 with accuracy value: 0.9783298373222351.
Better model found at epoch 4 with accuracy value: 0.9984143972396851.
Better model found at epoch 7 with accuracy value: 0.9994714856147766.


In [16]:
filename,gt=[],[]
test='/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/val/'
k=0
for j in ['fractured','not fractured']:
    temp=test+j
    for i in os.listdir(temp):
        temp1 = temp+'/'+i
        filename.append(temp1)
        gt.append(k)
    k = k+1

In [17]:
test_dl = dls.test_dl(filename)
preds, y = learner.get_preds(dl=test_dl)
predd = [torch.argmax(preds[i]).item() for i in range(len(preds))]
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(gt,predd,digits=4))
print(confusion_matrix(gt,predd))

              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       360
           1     1.0000    1.0000    1.0000       240

    accuracy                         1.0000       600
   macro avg     1.0000    1.0000    1.0000       600
weighted avg     1.0000    1.0000    1.0000       600

[[360   0]
 [  0 240]]


In [18]:
#Removing the last fully connected layer
resnet_features = torch.nn.Sequential(*list(resnet_model.children())[:-1])

In [19]:
resnet_features.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act3): ReLU(in

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_features = resnet_features.to(device)  # Move model to CPU or GPU

In [21]:
def extract_features(model, dataloader, device):
    features = []
    labels = []
    for inputs, targets in dataloader:
        inputs = inputs.to(device)
        with torch.no_grad():
            outputs = model(inputs)
        features.append(outputs.cpu().numpy())  # Move outputs back to CPU
        labels.append(targets.cpu().numpy())    # Move targets back to CPU
    features = np.concatenate(features)
    labels = np.concatenate(labels)
    return features, labels

In [22]:
for inputs, targets in train_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)
    
for inputs, targets in test_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)

In [23]:
train_features, train_labels = extract_features(resnet_features, train_loader, device)
test_features, test_labels = extract_features(resnet_features, test_loader, device)

#Flattening the features
train_features_flat = train_features.reshape(train_features.shape[0], -1)
test_features_flat = test_features.reshape(test_features.shape[0], -1)

In [24]:
#Feature scaling (necessary for KNN)
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features_flat)
test_features_scaled = scaler.transform(test_features_flat)

In [25]:
# K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_features_scaled, train_labels)
knn_pred = knn.predict(test_features_scaled)

knn_accuracy = accuracy_score(test_labels, knn_pred)
knn_precision = precision_score(test_labels, knn_pred, average='weighted')
knn_recall = recall_score(test_labels, knn_pred, average='weighted')
print("KNN Accuracy:", knn_accuracy)
print("KNN Precision:", knn_precision)
print("KNN Recall:", knn_recall)

KNN Accuracy: 1.0
KNN Precision: 1.0
KNN Recall: 1.0


In [26]:
# Naive Bayes Classifier (NBC)
nb = GaussianNB()
nb.fit(train_features_flat, train_labels)
nb_pred = nb.predict(test_features_flat)

nb_accuracy = accuracy_score(test_labels, nb_pred)
nb_precision = precision_score(test_labels, nb_pred, average='weighted')
nb_recall = recall_score(test_labels, nb_pred, average='weighted')

print("Naive Bayes Accuracy:", nb_accuracy)
print("Naive Bayes Precision:", nb_precision)
print("Naive Bayes Recall:", nb_recall)

Naive Bayes Accuracy: 1.0
Naive Bayes Precision: 1.0
Naive Bayes Recall: 1.0


**Part 3:** Using Scratch CNN as the feature extractor

In [27]:
#CNN Network
class ConvNetNN(nn.Module):
    def __init__(self,num_classes):
        super(ConvNetNN,self).__init__()
        
        #Layer1
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1) #conv2d filter
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU() 
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        
        #Layer2
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
        
        #Layer3
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        
        #Fully connected layer
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
       
    
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        
        output=output.view(-1,32*75*75)
                
        output=self.fc(output)
            
        return output

In [28]:
save_callback = SaveModelCallback(monitor='accuracy', fname='good_model')

In [29]:
learner2= Learner(dls = dls, model = ConvNetNN(2), model_dir = "/kaggle/working/", metrics = [accuracy, error_rate], cbs=save_callback, loss_func=CrossEntropyLossFlat(), opt_func=Adam)

In [30]:
learner2.fit_one_cycle(50, 0.001)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.412463,0.462632,0.788584,0.211416,00:18
1,0.228416,0.240646,0.906448,0.093552,00:18
2,0.213063,0.234958,0.912791,0.087209,00:18
3,0.171053,0.282416,0.922833,0.077167,00:18
4,0.304815,0.617100,0.909619,0.090381,00:18
5,0.276912,0.451542,0.910677,0.089323,00:18
6,0.451917,1.381917,0.857294,0.142706,00:18
7,0.383530,0.826432,0.896934,0.103066,00:18
8,0.388753,0.720874,0.913848,0.086152,00:18
9,0.342787,0.741919,0.904863,0.095137,00:18


Better model found at epoch 0 with accuracy value: 0.788583517074585.
Better model found at epoch 1 with accuracy value: 0.9064481854438782.
Better model found at epoch 2 with accuracy value: 0.9127907156944275.
Better model found at epoch 3 with accuracy value: 0.9228329658508301.
Better model found at epoch 10 with accuracy value: 0.9286469221115112.
Better model found at epoch 11 with accuracy value: 0.9365750551223755.
Better model found at epoch 13 with accuracy value: 0.9418604373931885.
Better model found at epoch 17 with accuracy value: 0.9513742327690125.
Better model found at epoch 20 with accuracy value: 0.954016923904419.
Better model found at epoch 26 with accuracy value: 0.957716703414917.
Better model found at epoch 28 with accuracy value: 0.9593023061752319.
Better model found at epoch 36 with accuracy value: 0.9603593945503235.
Better model found at epoch 37 with accuracy value: 0.961416482925415.
Better model found at epoch 46 with accuracy value: 0.9619450569152832.


In [31]:
filename,gt=[],[]
test='/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/val/'
k=0
for j in ['fractured','not fractured']:
    temp=test+j
    for i in os.listdir(temp):
        temp1 = temp+'/'+i
        filename.append(temp1)
        gt.append(k)
    k = k+1

In [32]:
test_dl = dls.test_dl(filename)
preds, y = learner2.get_preds(dl=test_dl)
predd = [torch.argmax(preds[i]).item() for i in range(len(preds))]
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(gt,predd,digits=4))
print(confusion_matrix(gt,predd))

              precision    recall  f1-score   support

           0     1.0000    0.9972    0.9986       360
           1     0.9959    1.0000    0.9979       240

    accuracy                         0.9983       600
   macro avg     0.9979    0.9986    0.9983       600
weighted avg     0.9983    0.9983    0.9983       600

[[359   1]
 [  0 240]]


In [33]:
state_dict = torch.load('/kaggle/working/good_model.pth', map_location=torch.device('cuda'))
state_dict = {k: v for k, v in state_dict.items() if 'fc' not in k}

In [34]:
#CNN Network
class ConvNet(nn.Module):
    def __init__(self,num_classes):
        super(ConvNet,self).__init__()
        
        #Layer1
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1) #conv2d filter
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU() 
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        
        #Layer2
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
        
        #Layer3
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        
    
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
                    
        return output

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
convnet_features = ConvNet(2).to(device)

In [37]:
convnet_features.load_state_dict(state_dict)

<All keys matched successfully>

In [38]:
def extracting_features(dl, model):
    features_list = []
    labels_list = []
    for xb, yb in dl:
        features = model(xb)
        features_list.append(features.cpu().detach().numpy().reshape(features.size(0), -1))
        labels_list.append(yb.cpu().detach().numpy())
    return np.vstack(features_list), np.concatenate(labels_list)

In [39]:
#Extracting features from images in train and test sets
train_features, train_labels = extracting_features(dls.train, convnet_features)
test_features, test_labels = extracting_features(dls.valid, convnet_features)

In [40]:
#Feature scaling (necessary for KNN)
scaler = StandardScaler()
train_features_scaled_2 = scaler.fit_transform(train_features)
test_features_scaled_2 = scaler.transform(test_features)

In [41]:
# K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_features_scaled_2, train_labels)
knn_pred = knn.predict(test_features_scaled_2)

knn_accuracy = accuracy_score(test_labels, knn_pred)
knn_precision = precision_score(test_labels, knn_pred, average='weighted')
knn_recall = recall_score(test_labels, knn_pred, average='weighted')
print("KNN Accuracy:", knn_accuracy)
print("KNN Precision:", knn_precision)
print("KNN Recall:", knn_recall)

KNN Accuracy: 0.7965116279069767
KNN Precision: 0.8506977629260825
KNN Recall: 0.7965116279069767


In [42]:
# Naive Bayes Classifier (NBC)
nb = GaussianNB()
nb.fit(train_features, train_labels)
nb_pred = nb.predict(test_features)

nb_accuracy = accuracy_score(test_labels, nb_pred)
nb_precision = precision_score(test_labels, nb_pred, average='weighted')
nb_recall = recall_score(test_labels, nb_pred, average='weighted')

print("Naive Bayes Accuracy:", nb_accuracy)
print("Naive Bayes Precision:", nb_precision)
print("Naive Bayes Recall:", nb_recall)

Naive Bayes Accuracy: 0.7579281183932347
Naive Bayes Precision: 0.7653291425976513
Naive Bayes Recall: 0.7579281183932347
